In [ ]:
import glob

new_pair_files = glob.glob('../../../ether_new_transactions/new_pairs*')
liquidity_update_files = glob.glob('../../../ether_new_transactions/liquidity_updates*')

In [ ]:
import pandas as pd

def load_dfs(files):
    dfs = []
    for f in files:
        df = pd.read_csv(f)
        if df.shape[0] > 0:
            df.loc[:, 'filename'] = f.split('/')[-1]
            dfs.append(df)
    return pd.concat(
        dfs, axis=0, ignore_index=True
    )

In [ ]:
import pandas as pd

new_pairs = load_dfs(new_pair_files)
liquidity_updates = load_dfs(liquidity_update_files)

In [ ]:
new_pairs.columns = list(map(lambda x: x.strip(), new_pairs.columns))
liquidity_updates.columns = list(map(lambda x: x.strip(), liquidity_updates.columns))

In [ ]:
liquidity_updates.token = liquidity_updates.token.apply(lambda x: x.strip())

In [ ]:
new_pairs.loc[:, 'old_time'] = new_pairs.time
liquidity_updates.loc[:, 'old_time'] = liquidity_updates.time

In [ ]:
def convert_time(x):
    try:
        return pd.Timestamp(x)
    except:
        return None

In [ ]:
new_pairs.time = new_pairs.time.apply(convert_time)
liquidity_updates.time = liquidity_updates.time.apply(convert_time)

In [ ]:
assert liquidity_updates.loc[liquidity_updates.time.isnull()].shape[0] == 0
assert new_pairs.loc[new_pairs.time.isnull()].shape[0] == 0

In [ ]:
liquidity_max_min_time = liquidity_updates.groupby('token').agg({'time': ['min', 'max']})
liquidity_max_min_time.columns = ['time_min', 'time_max']

In [ ]:
liquidity_max_min_time.loc[:, 'time_diff'] = (liquidity_max_min_time.time_max - liquidity_max_min_time.time_min).dt.total_seconds()

In [ ]:
num_minutes = 30
transaction_thresh = 5

In [ ]:
long_lived_tokens = liquidity_max_min_time.loc[liquidity_max_min_time.time_diff > (60 * num_minutes)].index.values

In [ ]:
def calculate_delta(liq_df, new_pairs_df, num_minutes):
    liq_df = liq_df.sort_values('time')
    liq_df.loc[:, 'init_eth_tok_ratio'] = -1
    token = liq_df.token.iloc[0]
    row = new_pairs_df.loc[new_pairs_df.token == token].iloc[0]
    if row.ether_token_ratio != -1 and row.ether_token_ratio != ' NaN':
        init_eth_liq = row.ether_liquidity
        init_tok_liq = row.token_liquidity
        init_time = row.time
    else:
        bool_filter = (liq_df.token_liquidity != 0) & (liq_df.ether_liquitity != 0)
        if sum(bool_filter == 0):
            return liq_df.iloc[:0]
        row = liq_df.loc[bool_filter].iloc[0]
        init_eth_liq = row.ether_liquitity
        init_tok_liq = row.token_liquidity
        init_time = row.time

    liq_df = liq_df.loc[liq_df.time.between(init_time, init_time + pd.to_timedelta(num_minutes, unit='m'))]
    if liq_df.shape[0] == 0:
        return liq_df.iloc[:0]
    liq_df.loc[:, 'init_eth_tok_ratio'] = init_eth_liq / init_tok_liq
    liq_df.loc[:, 'eth_tok_ratio_delta'] = liq_df.ether_token_ratio / liq_df.init_eth_tok_ratio
    liq_df.loc[liq_df.num_transactions < transaction_thresh, 'eth_tok_ratio_delta'] = 0
    liq_df.loc[:, 'time_delta'] = (liq_df.time - init_time).dt.total_seconds()
    liq_df.loc[:, 'eth_tok_ratio_delta_max_time'] = liq_df.loc[liq_df.eth_tok_ratio_delta.idxmax(), 'time_delta']
    liq_df.loc[:, 'transaction_rate'] = liq_df.num_transactions / (liq_df.time_delta + 1)
    return liq_df
        

In [ ]:
liquidity_udpates_enhanced = liquidity_updates.groupby('token').apply(lambda df: calculate_delta(df, new_pairs, num_minutes)).reset_index(drop=True)

In [ ]:
liquidity_aggs = liquidity_udpates_enhanced.groupby('token').agg(
    {
        'token_name': 'first', 'token_symbol': ['first', 'count'],  
        'eth_tok_ratio_delta': ['min', 'mean', 'max'], 'time_delta': ['min', 'max', 'mean'],
        'transaction_rate': ['mean', 'max'], 'time': 'first',
        'eth_tok_ratio_delta_max_time': 'first'
    }
)

In [ ]:
liquidity_aggs.columns = ['_'.join(col) if col[1] != 'first' else col[0] for col in liquidity_aggs.columns]

In [ ]:
liquidity_udpates_enhanced.to_csv(f'transaction_data/liquidity_updates_enh.csv', index=False)

In [ ]:
liquidity_aggs.to_csv('transaction_data/liquidity_updates_agg.csv', index=False)

In [ ]:
new_pairs.to_csv('transaction_data/new_pairs.csv', index=False)